In [1]:
from __future__ import print_function
from collections import defaultdict
import numpy as np
import scipy as sp
import cPickle as pickle
import time
from collections import defaultdict

from util_gradient import *

In [2]:
def init_theta(K, num_users, num_items):
    init_file = "alpha_beta_init_lam0.5_iter50.feature"
    alpha, beta_users, beta_items = pickle.load(open(init_file, "rb"))
    # alpha = np.mean(train_rating_array[:, 2])
    # beta_users = np.random.normal(0, 0.5, (num_users, ))
    # beta_items = np.random.normal(0, 0.5, (num_items, ))
    gamma_users = np.random.normal(0, 0.005, (num_users, K))
    gamma_items = np.random.normal(0, 0.005, (num_items, K))
    
    theta_length = (1 + 
                    beta_users.size + 
                    beta_items.size +
                    gamma_users.size + 
                    gamma_items.size)
    
    theta = np.empty((theta_length, ))
    return pack(theta, K, alpha, beta_users, beta_items, gamma_users, gamma_items)

In [3]:
def progress_callback(theta):
    def test_and_get_mse(rating_array, theta, K):
        ratings_predict = [predict_one_rating(user_index, item_index, theta, K) 
                           for user_index, item_index in rating_array[:, :2]]
        ratings = rating_array[:, 2]
        return get_mse(ratings_predict, ratings)
    
    print("train mse:", test_and_get_mse(train_rating_array, theta, K))
    print("valid mse:", test_and_get_mse(valid_rating_array, theta, K))

In [4]:
from scipy.optimize import minimize, fmin_l_bfgs_b

# load preprocessed data
# id <-> index infastructure

(user_id_map_index, 
 user_index_map_id,
 item_id_map_index,
 item_index_map_id) = pickle.load(open("id_index_map.feature", "rb"))
# train_rating_array, valid_rating_array
(train_rating_array, 
 valid_rating_array, 
 all_rating_array) = pickle.load(open("train_valid_all_rating_array.feature", "rb"))

In [5]:
# global variableds
K = 1
lam = 3.0
num_users = len(user_index_map_id)
num_items = len(item_index_map_id)
assert num_users == 35736
assert num_items == 37801

# init theta and grad_buffer
theta = init_theta(K, num_users, num_items)
grad_buffer = np.zeros_like(theta)

In [6]:
res = minimize(objective, theta, 
               args       = (grad_buffer, all_rating_array, lam, K), # train_rating_array
               method     = 'L-BFGS-B',
               jac        = gradient_only_alpha,
               options    = {'disp': True, 'maxiter': 3},
               callback   = progress_callback)
theta = res.x

train mse: 0.589261271889
valid mse: 0.594184883783
train mse: 0.592710029355
valid mse: 0.598084917106
train mse: 0.595521807665
valid mse: 0.601138040698
train mse: 0.595404463155
valid mse: 0.601136447227


In [7]:
res = minimize(objective, theta, 
               args       = (grad_buffer, all_rating_array, lam, K), # train_rating_array
               method     = 'L-BFGS-B',
               jac        = gradient_only_beta,
               options    = {'disp': True, 'maxiter': 3},
               callback   = progress_callback)
theta = res.x

train mse: 0.595090700883
valid mse: 0.600948698733
train mse: 0.594682050243
valid mse: 0.600431202373
train mse: 0.594430766763
valid mse: 0.60007444251
train mse: 0.594272910962
valid mse: 0.599849085081


In [8]:
res = minimize(objective, theta, 
               args       = (grad_buffer, all_rating_array, lam, K), # train_rating_array
               method     = 'L-BFGS-B',
               jac        = gradient_only_gamma_user,
               options    = {'disp': True, 'maxiter': 3},
               callback   = progress_callback)
theta = res.x

In [9]:
res = minimize(objective, theta, 
               args       = (grad_buffer, all_rating_array, lam, K), # train_rating_array
               method     = 'L-BFGS-B',
               jac        = gradient_only_alpha,
               options    = {'disp': True, 'maxiter': 3},
               callback   = progress_callback)
theta = res.x

train mse: 0.594031537568
valid mse: 0.599496905624
train mse: 0.595274816761
valid mse: 0.600820454029
train mse: 0.595253411301
valid mse: 0.600845525518


In [10]:
res = minimize(objective, theta, 
               args       = (grad_buffer, all_rating_array, lam, K), # train_rating_array
               method     = 'L-BFGS-B',
               jac        = gradient_only_beta,
               options    = {'disp': True, 'maxiter': 3},
               callback   = progress_callback)
theta = res.x

train mse: 0.595198066327
valid mse: 0.600841900792
train mse: 0.595093407895
valid mse: 0.600753945053
train mse: 0.594914558941
valid mse: 0.600653403217
train mse: 0.594807989232
valid mse: 0.600460957652


In [11]:
res = minimize(objective, theta, 
               args       = (grad_buffer, all_rating_array, lam, K), # train_rating_array
               method     = 'L-BFGS-B',
               jac        = gradient_only_gamma_item,
               options    = {'disp': True, 'maxiter': 3},
               callback   = progress_callback)
theta = res.x

In [12]:
res = minimize(objective, theta, 
               args       = (grad_buffer, all_rating_array, lam, K), # train_rating_array
               method     = 'L-BFGS-B',
               jac        = gradient,
               options    = {'disp': True, 'maxiter': 3},
               callback   = progress_callback)
theta = res.x

train mse: 0.594711776163
valid mse: 0.600322940578
train mse: 0.594688040704
valid mse: 0.600313815116
train mse: 0.594664161438
valid mse: 0.6003006511
train mse: 0.594668907458
valid mse: 0.60031710554


In [16]:
theta_mackup = np.copy(theta)

In [15]:
# get header_str and user_item_ids to predict
with open('pairs_Rating.txt') as f:
    # read and strip lines
    lines = [l.strip() for l in f.readlines()]
    # stirip out the headers
    header_str = lines.pop(0)
    # get a list of user_item_ids
    user_item_ids = [l.split('-') for l in lines]

# write to output file
f = open('predictions_Rating.txt', 'w')
print(header_str, file=f)
for user_id, item_id in user_item_ids:
    rating = predict_one_rating(user_id_map_index[user_id], 
                                item_id_map_index[item_id], 
                                theta, K)
    rating = min(5.0, rating)
    rating = max(0.0, rating)
    print('%s-%s,%s' % (user_id, item_id, rating), file=f)
f.close()